In [5]:
import cv2
import numpy as np

# Load your test image
img = cv2.imread('Udklip.png')
img_copy = img.copy()

# 1. Define your detected corner points (in the image)
image_points = np.array([
    [216, 58],   # top-left
    [1061, 39],  # top-right
    [1052, 667], # bottom-right
    [235, 646]   # bottom-left
], dtype=np.float32)

# 2. Define the desired rectangle (top-down)
grid_width, grid_height = 300, 300
grid_points = np.array([
    [0, 0],
    [grid_width, 0],
    [grid_width, grid_height],
    [0, grid_height]
], dtype=np.float32)

# 3. Compute homography
H, _ = cv2.findHomography(image_points, grid_points)
H_inv, _ = cv2.findHomography(grid_points, image_points)  # inverse for projecting grid back

# 4. Warp the image to get a top-down view
warped = cv2.warpPerspective(img, H, (grid_width, grid_height))

# 5. Draw grid on warped image
num_cells = 50
for i in range(1, num_cells):
    x = int(i * grid_width / num_cells)
    y = int(i * grid_height / num_cells)
    cv2.line(warped, (x, 0), (x, grid_height), (0, 255, 0), 1)
    cv2.line(warped, (0, y), (grid_width, y), (0, 255, 0), 1)

# 6. Project warped grid lines back onto original image
for i in range(1, num_cells):
    # Vertical lines
    pt1 = np.array([[i * grid_width / num_cells, 0]], dtype=np.float32)
    pt2 = np.array([[i * grid_width / num_cells, grid_height]], dtype=np.float32)
    orig_pt1 = cv2.perspectiveTransform(np.array([pt1]), H_inv)[0][0]
    orig_pt2 = cv2.perspectiveTransform(np.array([pt2]), H_inv)[0][0]
    cv2.line(img_copy, tuple(orig_pt1.astype(int)), tuple(orig_pt2.astype(int)), (0, 255, 0), 1)

    # Horizontal lines
    pt1 = np.array([[0, i * grid_height / num_cells]], dtype=np.float32)
    pt2 = np.array([[grid_width, i * grid_height / num_cells]], dtype=np.float32)
    orig_pt1 = cv2.perspectiveTransform(np.array([pt1]), H_inv)[0][0]
    orig_pt2 = cv2.perspectiveTransform(np.array([pt2]), H_inv)[0][0]
    cv2.line(img_copy, tuple(orig_pt1.astype(int)), tuple(orig_pt2.astype(int)), (0, 255, 0), 1)

# 7. Also draw "normal" corner interpolation grid directly (optional comparison)
for i in range(1, num_cells):
    alpha = i / num_cells

    # Interpolate between sides
    top = (1 - alpha) * image_points[0] + alpha * image_points[1]
    bottom = (1 - alpha) * image_points[3] + alpha * image_points[2]
    left = (1 - alpha) * image_points[0] + alpha * image_points[3]
    right = (1 - alpha) * image_points[1] + alpha * image_points[2]

    # Draw direct interpolation grid lines in RED
    cv2.line(img_copy, tuple(top.astype(int)), tuple(bottom.astype(int)), (0, 0, 255), 1)
    cv2.line(img_copy, tuple(left.astype(int)), tuple(right.astype(int)), (0, 0, 255), 1)

# 8. Show both views
cv2.imshow("Original with Homography Grid (Green) + Interpolated Grid (Red)", img_copy)
cv2.imshow("Warped Top-Down View with Grid", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()
